In [89]:
# This Notebook contains code for a simple ReAct Agent created using Langgraph.
# This is the same agent as we created from scratch here in the below link.
# https://github.com/karthikyerrakota/Generative-AI/blob/main/AI%20Agents/ReAct_agent_from_scratch.ipynb 

### Library imports

In [41]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, HumanMessage, SystemMessage, AIMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from dotenv import load_dotenv
import os

### Adding Persistance to Agent

In [19]:
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()

In [14]:
class AgentState(TypedDict):
    messages : Annotated[list[AnyMessage],operator.add]

### Agent Tools

In [33]:
@tool
def calculate(what):
    '''Calculates the given expression'''
    return eval(what)

@tool
def get_employee_salary(employee_name):
    '''Takes the employee name and return the salary'''
    
    if 'rahul' in employee_name.lower():
        return '40'
    else:
        return '20'

@tool
def get_employee_name(employee_id):
    '''Takes the id of the employee and provide the Name'''
    
    if 'ad241' in employee_id.lower():
        return 'Rahul'
    else:
        return 'other'

tool1 = calculate
tool2 = get_employee_salary
tool3 = get_employee_name

tools = [tool1,tool2,tool3]

### Agent

In [62]:
class Agent():

    def __init__(self,model,tools,checkpointer,system_message = ""):
        self.system_message = system_message
        graph = StateGraph(AgenState)
        graph.add_node("llm",self.openai_node)
        graph.add_node("action",self.take_action)
        graph.add_conditional_edges("llm",
                                   self.exisits_action_node,
                                   {True:"action",False:END})
        graph.add_edge("action",
                      "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(checkpointer=checkpointer)
        self.tools = {t.name:t for t in tools}
        self.model = model.bind_tools(tools)

    def exisits_action_node(self,state : AgenState):
        result = state['messages'][-1]
        return len(result.tool_calls)>0

    def openai_node(self, state : AgentState):
        message = state['messages']
        if self.system_message:
            l_messages = [SystemMessage(content = self.system_message)] + message
        output_message = self.model.invoke(l_messages)
        return {'messages':[output_message]}

    def take_action(self, state : AgenState):
        tool_calls = state['messages'][-1].tool_calls
        print("TOOL CALLS......",tool_calls)
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            if not t['name'] in self.tools:      # check for bad tool name from LLM
                print("\n ....bad tool name....")
                result = "bad tool name, retry"  # instruct LLM to retry if bad
            else:
                result = self.tools[t['name']].invoke(t['args'])

            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [63]:
system_prompt = '''You operate in a loop of Thought, Action, PAUSE, Observation.  
At the end of the loop, you output an Answer.  

Use Thought to describe your reasoning about the question you've been asked.  
Use Action to execute one of the available actions and then return PAUSE.  
Observation will be the result of running that action.  

While answering First try to look at the given tools only.

Your available actions are:  

1. **calculate**:  
   e.g., calculate: (5 + 3) * 2  
   Executes a mathematical operation and returns the result.  

2. **get_employee_salary**:  
   e.g., get_employee_salary: 30  
   Takes the employee name and return the salary.  

3. **get_employee_name**:  
   e.g., get_employee_name: karthik  
   Takes the id of the employee and provide the Name.  

### Example Session:  

Question: What is the salary of Ritu?  
Thought: I need to look up the Salary of Ritu.  
Action: get_employee_salary: Ritu  
PAUSE

When called again with:  

Observation: The salary of Ritu is 20.  

You then output:  
Answer: The salary of Ritu is 20.  

Question: What is 12 * 8 + 5?  
Thought: I should perform this calculation using the calculate tool.  
Action: calculate: 12 * 8 + 5  
PAUSE 

You will be called again with:  

Observation: The result is 101.  

You then output:  
Answer: The result of 12 * 8 + 5 is 101.  

'''.strip()

### Invoking Agent

In [88]:
messages = [HumanMessage(content="What is the average salary of Rahul and karthik?")]

# Load environment variables from .env file.
load_dotenv()
# load your openai api key from .env file.
openai_key = os.getenv("API_KEY")

model = ChatOpenAI(model="gpt-3.5-turbo",api_key=openai_key)
abot = Agent(model, tools, checkpointer=memory, system_message=system_prompt)
thread = {"configurable":{"thread_id":"2"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

print()
print("ANSWER : ",list(event.values())[0]['messages'][0].content)

{'messages': [AIMessage(content='Thought: To calculate the average salary of Rahul and Karthik, I need to find the salaries of both employees.\n\nAction: multi_tool_use.parallel\n{\n  tool_uses: [\n    {\n      recipient_name: "functions.get_employee_salary",\n      parameters: { "employee_name": "Rahul" }\n    },\n    {\n      recipient_name: "functions.get_employee_salary",\n      parameters: { "employee_name": "Karthik" }\n    }\n  ]\n}\nPAUSE', additional_kwargs={'tool_calls': [{'id': 'call_X7SiVZ6RKU3mNMKip1l1qFt9', 'function': {'arguments': '{"employee_name": "Rahul"}', 'name': 'get_employee_salary'}, 'type': 'function'}, {'id': 'call_5AQjiQqGHjqPuybcQnWLYHRW', 'function': {'arguments': '{"employee_name": "Karthik"}', 'name': 'get_employee_salary'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 153, 'prompt_tokens': 821, 'total_tokens': 974, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoni